# Data Validation for Kaggle Tabular Playground Series Mar 2021 Using Python and TensorFlow Data Validation
### David Lowe
### June 8, 2021

SUMMARY: The project aims to construct a data validation flow using TensorFlow Data Validation (TFDV) and document the end-to-end steps using a template. The Kaggle Tabular Playground Series Mar 2021 dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

INTRODUCTION: Kaggle wants to provide an approachable environment for relatively new people in their data science journey. Since January 2021, they have hosted playground-style competitions on Kaggle with fun but less complex, tabular datasets. The dataset may be synthetic but is based on a real dataset and generated using a CTGAN. The original dataset tries to predict the amount of an insurance claim. Although the features are anonymized, they have properties relating to real-world features.

Additional Notes: I adapted this workflow from the TensorFlow Data Validation tutorial on TensorFlow.org (https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic). I also plan to build a TFDV script for validating future datasets and building machine learning models.

CONCLUSION: In this iteration, the data validation workflow helped to validate the features and structures of the training, validation, and test datasets. The workflow also generated statistics over different slices of data which can help track model and anomaly metrics.

Dataset Used: Kaggle Tabular Playground 2021 Mar Data Set

Dataset ML Model: Binary classification with numerical and categorical attributes

Dataset Reference: https://www.kaggle.com/c/tabular-playground-series-mar-2021

Data validation for a machine learning project generally can be broken down into the following tasks:

1. Prepare Environment
2. Generate and Visualize Training Data Statistics
3. Check Anomalies in Validation Dataset
4. Check Anomalies in Test Dataset
5. Check for Data Drift and Skew
6. Display Stats for Data Slices
7. Finalize the Schema

## Task 1 - Prepare Environment

### 1.a) Load libraries and modules

In [1]:
# Set the random seed number for reproducible results
RNG_SEED = 8

In [2]:
# Import packages
import os
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tempfile, urllib, zipfile
import tensorflow_data_validation as tfdv
from tensorflow.python.lib.io import file_io
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics
from tensorflow_metadata.proto.v0 import schema_pb2

### 1.b) Set up the controlling parameters and functions

In [3]:
# Begin the timer for the script processing
start_time_script = datetime.now()

# Set the percentage sizes for splitting the dataset
TEST_SET_RATIO = 0.5
VAL_SET_RATIO = 0.2

# Set TF's logger to only display errors to avoid internal warnings being shown
tf.get_logger().setLevel('ERROR')

### 1.c) Load dataset

In [4]:
# Read CSV data into a dataframe and mark the missing data that is encoded with '?' string as NaN
dataset_path = 'https://dainesanalytics.com/datasets/kaggle-tabular-playground-2021mar/train.csv'
df_dataset_import = pd.read_csv(dataset_path, header=0, na_values = '?')

# Take a peek at the dataframe after import
print(df_dataset_import.head())

   id cat0 cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8  ...     cont2     cont3  \
0   0    A    I    A    B    B   BI    A    S    Q  ...  0.759439  0.795549   
1   1    A    I    A    A    E   BI    K    W   AD  ...  0.386385  0.541366   
2   2    A    K    A    A    E   BI    A    E   BM  ...  0.343255  0.616352   
3   3    A    K    A    C    E   BI    A    Y   AD  ...  0.831147  0.807807   
4   4    A    I    G    B    E   BI    C    G    Q  ...  0.338818  0.277308   

      cont4     cont5     cont6     cont7     cont8     cont9    cont10 target  
0  0.681917  0.621672  0.592184  0.791921  0.815254  0.965006  0.665915      0  
1  0.388982  0.357778  0.600044  0.408701  0.399353  0.927406  0.493729      0  
2  0.793687  0.552877  0.352113  0.388835  0.412303  0.292696  0.549452      0  
3  0.800032  0.619147  0.221789  0.897617  0.633669  0.760318  0.934242      0  
4  0.610578  0.128291  0.578764  0.279167  0.351103  0.357084  0.328960      1  

[5 rows x 32 columns]


In [5]:
df_dataset_import.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      300000 non-null  int64  
 1   cat0    300000 non-null  object 
 2   cat1    300000 non-null  object 
 3   cat2    300000 non-null  object 
 4   cat3    300000 non-null  object 
 5   cat4    300000 non-null  object 
 6   cat5    300000 non-null  object 
 7   cat6    300000 non-null  object 
 8   cat7    300000 non-null  object 
 9   cat8    300000 non-null  object 
 10  cat9    300000 non-null  object 
 11  cat10   300000 non-null  object 
 12  cat11   300000 non-null  object 
 13  cat12   300000 non-null  object 
 14  cat13   300000 non-null  object 
 15  cat14   300000 non-null  object 
 16  cat15   300000 non-null  object 
 17  cat16   300000 non-null  object 
 18  cat17   300000 non-null  object 
 19  cat18   300000 non-null  object 
 20  cont0   300000 non-null  float64
 21  cont1   30

In [6]:
# Read CSV data into a dataframe and mark the missing data that is encoded with '?' string as NaN
dataset_path = 'https://dainesanalytics.com/datasets/kaggle-tabular-playground-2021mar/test.csv'
df_test = pd.read_csv(dataset_path, header=0, na_values = '?')

# Take a peek at the dataframe after import
print(df_test.head())

   id cat0 cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8  ...     cont1     cont2  \
0   5    A    F    A    A    F   BI    A   AH   AX  ...  0.735690  0.578366   
1   6    A    H    C    A    E   AB    I    F    N  ...  0.313703  0.928885   
2   8    A    N    C    A    F   AB    A   AH   BC  ...  0.448201  0.424876   
3   9    B    L    C    A    F   BI    A    E   AX  ...  0.666092  0.598943   
4  11    A    F    A    B    F   BI    A   AH    I  ...  0.772229  0.479572   

      cont3     cont4     cont5     cont6     cont7     cont8     cont9  \
0  0.723154  0.228037  0.356227  0.551249  0.655693  0.598331  0.359987   
1  0.516602  0.600169  0.795224  0.248987  0.654614  0.347944  0.565520   
2  0.344729  0.242073  0.270632  0.746740  0.335590  0.341238  0.252289   
3  0.561971  0.806347  0.735983  0.538724  0.381566  0.481660  0.348514   
4  0.767745  0.252454  0.354810  0.178920  0.763479  0.562491  0.466261   

     cont10  
0  0.947489  
1  0.388580  
2  0.411592  
3  0.325723  
4  0

### 1.d) Splitting Data into Sets

In [7]:
# Split the data further into training, validation, and test datasets
df_train, df_val = train_test_split(df_dataset_import, test_size=VAL_SET_RATIO, random_state=RNG_SEED)

print("Training dataset has {} records and {} columns.".format(df_train.shape[0], df_train.shape[1]))
print("Validation dataset has {} records and {} columns.".format(df_val.shape[0], df_val.shape[1]))
print("Test dataset has {} records and {} columns".format(df_test.shape[0], df_test.shape[1]))

Training dataset has 240000 records and 32 columns.
Validation dataset has 60000 records and 32 columns.
Test dataset has 200000 records and 31 columns


## Task 2 - Generate and Visualize Training Data Statistics

### 2.a) Removing Irrelevant Features

In [8]:
# Define features to remove
features_to_remove = {'id'}

# Collect features to whitelist while computing the statistics
features_to_keep = [col for col in df_dataset_import.columns if (col not in features_to_remove)]

# Instantiate a StatsOptions class and define the feature_whitelist property
options_train = tfdv.StatsOptions(feature_allowlist=features_to_keep)

# Review the features to generate the statistics
print(options_train.feature_allowlist)

['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'target']


### 2.b) Generate Statistics for Training Data

In [9]:
train_stats = tfdv.generate_statistics_from_dataframe(df_train, stats_options=options_train)

# get the number of features used to compute statistics
print(f"Number of features used: {len(train_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {train_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {train_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {train_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 31
Number of examples used: 240000
First feature: cat0
Last feature: target


### 2.c) Visualize Training Statistics

In [10]:
tfdv.visualize_statistics(train_stats)

### 2.d) Infer Training Schema

In [11]:
# Infer the data schema by using the training statistics previously generated
dataset_schema = tfdv.infer_schema(train_stats)

# Display the data schema
tfdv.display_schema(dataset_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'cat0',STRING,required,,'cat0'
'cat1',STRING,required,,'cat1'
'cat2',STRING,required,,'cat2'
'cat3',STRING,required,,'cat3'
'cat4',STRING,required,,'cat4'
'cat5',STRING,required,,'cat5'
'cat6',STRING,required,,'cat6'
'cat7',STRING,required,,'cat7'
'cat8',STRING,required,,'cat8'


/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'cat0',"'A', 'B'"
'cat1',"'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O'"
'cat2',"'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'Q', 'R', 'S', 'U'"
'cat3',"'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'N'"
'cat4',"'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T'"
'cat5',"'A', 'AA', 'AB', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'B', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BX', 'BY', 'C', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', 'ZZ'"
'cat6',"'A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'K', 'M', 'O', 'Q', 'S', 'U', 'W', 'Y'"
'cat7',"'A', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'B', 'BA', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y'"
'cat8',"'A', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'B', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'C', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'S', 'T', 'U', 'V', 'X', 'Y'"


## Task 3 - Check Anomalies in Validation Dataset

### 3.a) Generate Statistics for Validation Data

In [12]:
val_stats = tfdv.generate_statistics_from_dataframe(df_val, stats_options=options_train)

# get the number of features used to compute statistics
print(f"Number of features used: {len(val_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {val_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {val_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {val_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 31
Number of examples used: 60000
First feature: cat0
Last feature: target


### 3.b) Compare Validation with Training Statistics

In [13]:
tfdv.visualize_statistics(lhs_statistics=val_stats, rhs_statistics=train_stats,
                          lhs_name='VAL_DATASET', rhs_name='TRAIN_DATASET')

### 3.c) Detect Anomalies

In [14]:
val_anomalies = tfdv.validate_statistics(statistics=val_stats, schema=dataset_schema)
tfdv.display_anomalies(val_anomalies)

/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


### 3.d) Fix Validation Data Anomalies in Schema

In [15]:
# Not applicable in this iteration of the data validation exercise

## Task 4 - Check Anomalies in Test Dataset

### 4.a) Generate Statistics for Test Data

In [16]:
# Define a new statistics options by the tfdv.StatsOptions class for the serving data by passing the previously inferred schema
options_test = tfdv.StatsOptions(schema=dataset_schema, infer_type_from_schema=True, feature_allowlist=features_to_keep)

In [17]:
# Generate serving dataset statistics
test_stats = tfdv.generate_statistics_from_dataframe(df_test, stats_options=options_test)

# get the number of features used to compute statistics
print(f"Number of features used: {len(test_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {test_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {test_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {test_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 30
Number of examples used: 200000
First feature: cat0
Last feature: cont10


### 4.b) Compare Test with Training Statistics

In [18]:
test_anomalies = tfdv.validate_statistics(statistics=test_stats, schema=dataset_schema)
tfdv.display_anomalies(test_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'target',Column dropped,Column is completely missing


### 4.c) Fix Test Data Anomalies in Schema

In [19]:
# Not applicable in this iteration of the data validation exercise

### 4.d) Organize Schema based on Environment

In [20]:
# All features are by default in both TRAINING and SERVING environments.
dataset_schema.default_environment.append('TRAINING')
dataset_schema.default_environment.append('TESTING')

In [21]:
# Specify that 'target' feature is not in TEST environment.
tfdv.get_feature(dataset_schema, 'target').not_in_environment.append('TESTING')

# Re-calculate and re-display anomalies with the new environment parameters
test_anomalies = tfdv.validate_statistics(statistics=test_stats, schema=dataset_schema, environment='TESTING')
tfdv.display_anomalies(test_anomalies)

/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Task 5 - Check for Data Drift and Skew

In [22]:
# Not applicable in this iteration of the data validation exercise

## Task 6 - Display Stats for Data Slices

In [23]:
def split_datasets(dataset_list):
    '''
    split datasets.

            Parameters:
                    dataset_list: List of datasets to split

            Returns:
                    datasets: sliced data
    '''
    datasets = []
    for dataset in dataset_list.datasets:
        proto_list = DatasetFeatureStatisticsList()
        proto_list.datasets.extend([dataset])
        datasets.append(proto_list)
    return datasets

In [24]:
def display_stats_at_index(index, datasets):
    '''
    display statistics at the specified data index

            Parameters:
                    index : index to show the anomalies
                    datasets: split data

            Returns:
                    display of generated sliced data statistics at the specified index
    '''
    if index < len(datasets):
        print(datasets[index].datasets[0].name)
        tfdv.visualize_statistics(datasets[index])

In [25]:
def sliced_stats_for_slice_fn(slice_fn, approved_cols, dataframe, schema):
    '''
    generate statistics for the sliced data.

            Parameters:
                    slice_fn : slicing definition
                    approved_cols: list of features to pass to the statistics options
                    dataframe: pandas dataframe to slice
                    schema: the schema

            Returns:
                    slice_info_datasets: statistics for the sliced dataset
    '''
    # Set the StatsOptions
    slice_stats_options = tfdv.StatsOptions(schema=schema,
                                            slice_functions=[slice_fn],
                                            infer_type_from_schema=True,
                                            feature_allowlist=approved_cols)
    
    # Convert Dataframe to CSV since `slice_functions` works only with `tfdv.generate_statistics_from_csv`
    CSV_PATH = 'slice_sample.csv'
    dataframe.to_csv(CSV_PATH)
    
    # Calculate statistics for the sliced dataset
    sliced_stats = tfdv.generate_statistics_from_csv(CSV_PATH, stats_options=slice_stats_options)
    
    # Split the dataset using the previously defined split_datasets function
    slice_info_datasets = split_datasets(sliced_stats)
    
    return slice_info_datasets

In [26]:
# Generate slice function for one of the features
slice_fn = slicing_util.get_feature_value_slicer(features={'cat15': None})

# Generate stats for the sliced dataset
slice_datasets = sliced_stats_for_slice_fn(slice_fn, features_to_keep, dataframe=df_train, schema=dataset_schema)

# Print name of slices for reference
print(f'Statistics generated for:\n')
print('\n'.join([sliced.datasets[0].name for sliced in slice_datasets]))

# Display at index 1 as an example
display_stats_at_index(1, slice_datasets) 

Statistics generated for:

All Examples
cat15_D
cat15_B
cat15_A
cat15_C
cat15_D


## Task 7 - Finalize the Schema

In [27]:
# Create output directory
OUTPUT_DIR = "output"
file_io.recursive_create_dir(OUTPUT_DIR)

# Use TensorFlow text output format pbtxt to store the schema
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')

# write_schema_text function expect the defined schema and output path as parameters
tfdv.write_schema_text(dataset_schema, schema_file) 

In [28]:
print ('Total time for the script:',(datetime.now() - start_time_script))

Total time for the script: 0:01:57.113882
